In [1]:
import sys
from itertools import count
from torch import autograd
from torch_geometric.utils import dense_to_sparse
import copy

sys.path.append('../')
from models.gcn import *
from utils.datasets import *

In [1]:
import os
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))

CUDA_VISIBLE_DEVICES: 1,2,3,4,5,6,7,8,9,10


In [2]:
import torch

print("CUDA Device Count:", torch.cuda.device_count())

for i in range(torch.cuda.device_count()):
    print(f"Device {i}: {torch.cuda.get_device_name(i)}")

CUDA Device Count: 3
Device 0: NVIDIA H200 MIG 1g.18gb
Device 1: NVIDIA H200 MIG 1g.18gb
Device 2: NVIDIA H200 MIG 1g.18gb


In [3]:
!nvidia-smi

Sun Mar  2 21:52:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H200                    On  |   00000000:1C:00.0 Off |                   On |
| N/A   36C    P0             77W /  700W |     107MiB / 143771MiB |     N/A      Default |
|                                         |                        |              Enabled |
+-----------------------------------------+-----

In [4]:
device = "cuda:1" if torch.cuda.is_available() else "cpu"

In [5]:
from models.trainable import *

In [6]:
from meta_pgd import *

In [7]:
# dataset_directory = "../Cora"
cora_dataset = Planetoid(root='', name='Cora')
data = cora_dataset[0].to(device)
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [8]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [64]).to(device)

In [9]:
model.reset_parameters()
train = Trainable(model)
train.fit(data, 200)

Epoch 0, Train Loss - 1.9533783197402954, Val Loss - 1.8056105375289917, Val Accuracy - 0.623
Epoch 20, Train Loss - 0.010077104903757572, Val Loss - 0.6095793843269348, Val Accuracy - 0.801
Epoch 40, Train Loss - 0.010379408486187458, Val Loss - 0.6340352892875671, Val Accuracy - 0.803
Epoch 60, Train Loss - 0.015796003863215446, Val Loss - 0.5837293863296509, Val Accuracy - 0.818
Epoch 80, Train Loss - 0.012594752945005894, Val Loss - 0.6316797137260437, Val Accuracy - 0.8
Epoch 100, Train Loss - 0.01380453072488308, Val Loss - 0.5937012434005737, Val Accuracy - 0.817
Epoch 120, Train Loss - 0.013288289308547974, Val Loss - 0.6193312406539917, Val Accuracy - 0.81
Epoch 140, Train Loss - 0.01103497575968504, Val Loss - 0.6200186610221863, Val Accuracy - 0.81
Epoch 160, Train Loss - 0.010321670211851597, Val Loss - 0.6277612447738647, Val Accuracy - 0.811
Epoch 180, Train Loss - 0.010367141105234623, Val Loss - 0.6063375473022461, Val Accuracy - 0.816
Epoch 200, Train Loss - 0.00992275

In [ ]:
!nvidia-smi

# MetaPGD Tests

In [8]:
attacker = MetaPGD(data, device=device)

In [9]:
attacker.setup_surrogate(
    model,
    labeled_nodes=data.train_mask,
    unlabeled_nodes=data.test_mask,
    lambda_=0.0,
)

In [10]:
attacker.num_budgets

In [11]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [12]:
allocated_memory = torch.cuda.memory_allocated(device)
print(f"Current allocated memory: {allocated_memory / (1024**3):.2f} GB")

Current allocated memory: 0.13 GB


In [14]:
attacker.reset()
#losses,
best_loss, best_pert = attacker.attack(0.2,base_lr=0.01, grad_clip=1, iterations=400)

Running PGD Attack...:   0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

1055 tensor(1065, device='cuda:0')
1055 tensor(1054, device='cuda:0')
1055 tensor(1065, device='cuda:0')
1055 tensor(1054, device='cuda:0')
1055 tensor(1065, device='cuda:0')
1055 tensor(1054, device='cuda:0')
1055 tensor(1065, device='cuda:0')
1055 tensor(1054, device='cuda:0')
1055 tensor(1065, device='cuda:0')
1055 tensor(1054, device='cuda:0')
1055 tensor(1065, device='cuda:0')
1055 tensor(1054, device='cuda:0')
1055 tensor(1065, device='cuda:0')
1055 tensor(1054, device='cuda:0')
1055 tensor(1065, device='cuda:0')
1055 tensor(1054, device='cuda:0')
1055 tensor(1065, device='cuda:0')
1055 tensor(1054, device='cuda:0')
1055 tensor(1065, device='cuda:0')
1055 tensor(1054, device='cuda:0')
1055 tensor(1065, device='cuda:0')
1055 tensor(1054, device='cuda:0')
1055 tensor(1065, device='cuda:0')
1055 tensor(1054, device='cuda:0')
1055 tensor(1065, device='cuda:0')
1055 tensor(1054, device='cuda:0')
1055 tensor(1065, device='cuda:0')
1055 tensor(1054, device='cuda:0')
1055 tensor(1065, de

In [15]:
attacker.loss_list

[tensor(0.0269, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(0.1528, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(0.3311, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(0.6627, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(0.6281, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(0.7855, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(0.9303, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(1.0057, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(1.0864, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(1.1026, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(1.1333, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(1.1612, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(1.1964, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(1.2772, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(1.2269, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(1.2123, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(

In [16]:
new_data = handle_new_edges(data, attacker, device)

Updated edge index: torch.Size([2, 12664])


In [17]:
loss, acc = train.test(new_data)
print(f'Loss: {loss}, Acc: {acc}')

Loss: 0.6584149599075317, Acc: 0.793


In [18]:
loss, acc = train.test(data)
print(f'Loss: {loss}, Acc: {acc}')

Loss: 0.6534879207611084, Acc: 0.802


In [19]:
model2 = GCN(data.x.shape[1], cora_dataset.num_classes, [64]).to(device)

In [20]:
model2.reset_parameters()
train = Trainable(model2)
train.fit(new_data, 200)

Epoch 0, Train Loss - 1.9460113048553467, Val Loss - 1.848807692527771, Val Accuracy - 0.434
Epoch 20, Train Loss - 0.2179020792245865, Val Loss - 1.0081311464309692, Val Accuracy - 0.65
Epoch 40, Train Loss - 0.05260324478149414, Val Loss - 1.2403444051742554, Val Accuracy - 0.617
Epoch 60, Train Loss - 0.05846146121621132, Val Loss - 1.2221503257751465, Val Accuracy - 0.619
Epoch 80, Train Loss - 0.04941694810986519, Val Loss - 1.3108630180358887, Val Accuracy - 0.597
Epoch 100, Train Loss - 0.04585389792919159, Val Loss - 1.3905452489852905, Val Accuracy - 0.589
Epoch 120, Train Loss - 0.03984278440475464, Val Loss - 1.3938337564468384, Val Accuracy - 0.583
Epoch 140, Train Loss - 0.04062098264694214, Val Loss - 1.4042916297912598, Val Accuracy - 0.592
Epoch 160, Train Loss - 0.04401300847530365, Val Loss - 1.403091549873352, Val Accuracy - 0.599
Epoch 180, Train Loss - 0.048639800399541855, Val Loss - 1.4306875467300415, Val Accuracy - 0.598
Epoch 200, Train Loss - 0.03870876878499

# Meta Greedy Tests

In [8]:
from greedy_gd import *

In [9]:
device = 'cuda:1'

In [10]:
attacker = Metattack(data, device=device)
attacker.setup_surrogate(
    model,
    labeled_nodes=data.train_mask,
    unlabeled_nodes=data.test_mask,
    lambda_=0.5,
)

In [11]:
import torch

# Number of GPUs visible to PyTorch
print("CUDA Device Count:", torch.cuda.device_count())

# List all visible devices
for i in range(torch.cuda.device_count()):
    print(f"Device {i}: {torch.cuda.get_device_name(i)}")

CUDA Device Count: 4
Device 0: NVIDIA H200
Device 1: NVIDIA H200 MIG 1g.18gb
Device 2: NVIDIA H200 MIG 1g.18gb
Device 3: NVIDIA H200 MIG 1g.18gb


In [12]:
attacker.reset()
attacker.attack(1.00)

Peturbing graph...:   0%|          | 0/5278 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
new_data = handle_new_edges(data, attacker, device)

Updated edge index: torch.Size([2, 11078])


In [11]:
loss, acc = train.test(new_data)
print(f'Loss: {loss}, Acc: {acc}')

Loss: 0.6588292121887207, Acc: 0.799


In [12]:
loss, acc = train.test(data)
print(f'Loss: {loss}, Acc: {acc}')

Loss: 0.6605153679847717, Acc: 0.8


In [30]:
model2 = GCN(data.x.shape[1], cora_dataset.num_classes, [64]).to(device)

In [31]:
model2.reset_parameters()
train = Trainable(model2)
train.fit(new_data, 200)

Epoch 0, Train Loss - 1.946224570274353, Val Loss - 1.8570330142974854, Val Accuracy - 0.325
Epoch 20, Train Loss - 0.11124295741319656, Val Loss - 2.7840189933776855, Val Accuracy - 0.408
Epoch 40, Train Loss - 0.04103148356080055, Val Loss - 3.818260908126831, Val Accuracy - 0.412
Epoch 60, Train Loss - 0.03561371564865112, Val Loss - 3.7860445976257324, Val Accuracy - 0.411
Epoch 80, Train Loss - 0.03578553348779678, Val Loss - 3.9414350986480713, Val Accuracy - 0.417
Epoch 100, Train Loss - 0.035183899104595184, Val Loss - 4.1424689292907715, Val Accuracy - 0.407
Epoch 120, Train Loss - 0.030303457751870155, Val Loss - 4.432448387145996, Val Accuracy - 0.401
Epoch 140, Train Loss - 0.03577311709523201, Val Loss - 4.602138519287109, Val Accuracy - 0.399
Epoch 160, Train Loss - 0.021818237379193306, Val Loss - 4.818694591522217, Val Accuracy - 0.405
Epoch 180, Train Loss - 0.021507030352950096, Val Loss - 4.969539165496826, Val Accuracy - 0.405
Epoch 200, Train Loss - 0.0288426186889